In [21]:
# text from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print('---')
print(text)
print("length:", len(text))
print('---')
#print(tokens)
print("length:", len(tokens))

---
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
length: 533
---
length: 616


In [22]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # we iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1 # default to 0
    return counts

stats = get_stats(tokens)
# print(stats)
# print(sorted(((v,k) for k,v in stats.items()), reverse=True))

In [23]:
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [24]:
def merge(ids, pair, idx):
  # In the list of ints (ids), replace all consecutive occurences of pair with new token idx
  newids = []
  i = 0
  while i < len(ids):
    # if we are not at the very last position AND the pair matches, replace it
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

print(merge([5, 6, 6, 7, 9, 1], (6, 7), 199))

tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print("length:", len(tokens2))

[5, 6, 199, 9, 1]
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 256, 118, 101, 114, 121, 32, 110, 97, 109, 256, 115, 116, 114, 105, 107, 101, 115, 32, 102, 101, 97, 114, 32, 97, 110, 100, 32, 97, 119, 256, 105, 110, 116, 111, 32, 116, 104, 256, 104, 101, 97, 114, 116, 115, 32, 111, 102, 32, 112, 114, 111, 103, 114, 97, 109, 109, 101, 114, 115, 32, 119, 111, 114, 108, 100, 119, 105, 100, 101, 46, 32, 87, 256, 97, 108, 108, 32, 107, 110, 111, 119, 32, 119, 256, 111, 117, 103, 104, 116, 32, 116, 111,

In [25]:
# making the training text longer to have more representative token statistics
# text from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = """In this post we will describe and demystify the relevant artifacts in the paper “Attention is all you need” (Vaswani, Ashish & Shazeer, Noam & Parmar, Niki & Uszkoreit, Jakob & Jones, Llion & Gomez, Aidan & Kaiser, Lukasz & Polosukhin, Illia. (2017))[1]. This paper was a great advance in the use of the attention mechanism, being the main improvement for a model called Transformer. The most famous current models that are emerging in NLP tasks consist of dozens of transformers or some of their variants, for example, GPT-2 or BERT.

We will describe the components of this model, analyze their operation and build a simple model that we will apply to a small-scale NMT problem (Neural Machine Translation). To read more about the problem that we will address and to know how the basic attention mechanism works, I recommend you to read my previous post “A Guide on the Encoder-Decoder Model and the Attention Mechanism”.

Why we need Transformer
In sequence-to-sequence problems such as the neural machine translation, the initial proposals were based on the use of RNNs in an encoder-decoder architecture. These architectures have a great limitation when working with long sequences, their ability to retain information from the first elements was lost when new elements were incorporated into the sequence. In the encoder, the hidden state in every step is associated with a certain word in the input sentence, usually one of the most recent. Therefore, if the decoder only accesses the last hidden state of the decoder, it will lose relevant information about the first elements of the sequence. Then to deal with this limitation, a new concept were introduced the attention mechanism.

Instead of paying attention to the last state of the encoder as is usually done with RNNs, in each step of the decoder we look at all the states of the encoder, being able to access information about all the elements of the input sequence. This is what attention does, it extracts information from the whole sequence, a weighted sum of all the past encoder states. This allows the decoder to assign greater weight or importance to a certain element of the input for each element of the output. Learning in every step to focus in the right element of the input to predict the next output element.

But this approach continues to have an important limitation, each sequence must be treated one element at a time. Both the encoder and the decoder have to wait till the completion of t-1 steps to process thet-th step. So when dealing with huge corpus it is very time consuming and computationally inefficient.

What is the Transformer?
In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization … the Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution.

“Attention is all you need” paper [1]

The Transformer model extract features for each word using a self-attention mechanism to figure out how important all the other words in the sentence are w.r.t. to the aforementioned word. And no recurrent units are used to obtain this features, they are just weighted sums and activations, so they can be very parallelizable and efficient.

But we will dive deeper into its architecture (next figure) to understand what all this pieces do [1].


From “Attention is all you need” paper by Vaswani, et al., 2017 [1]
We can observe there is an encoder model on the left side and the decoder on the right one. Both contains a core block of “an attention and a feed-forward network” repeated N times. But first we need to explore a core concept in depth: the self-attention mechanism.

Self-Attention: the fundamental operation
Self-attention is a sequence-to-sequence operation: a sequence of vectors goes in, and a sequence of vectors comes out. Let’s call the input vectors x1, x2,…, xt and the corresponding output vectors y1, y2,…, yt. The vectors all have dimension k. To produce output vector yi, the self attention operation simply takes a weighted average over all the input vectors, the simplest option is the dot product.

Transformers from scratch by Peter Bloem [2]

In the self-attention mechanism of our model we need to introduce three elements: Queries, Values and Keys

The Query, The Value and The Key
Every input vector is used in three different ways in the self-attention mechanism: the Query, the Key and the Value. In every role, it is compared to the other vectors to get its own output yi(Query), to get the j-th output yj(Key) and to compute each output vector once the weights have been established (Value).

To obtain this roles, we need three weight matrices of dimensions k x k and compute three linear transformation for each xi:


“Transformers from scratch” by Peter Bloem [2]
These three matrices are usually known as K, Q and V, three learnable weight layers that are applied to the same encoded input. Consequently, as each of these three matrices come from the same input, we can apply the attention mechanism of the input vector with itself, a “self-attention”.

The Scaled Dot-Product Attention
The input consists of queries and keys of dimension dk, and values of dimension dv. We compute the dot product of the query with all keys, divide each by the square root of dk, and apply a softmax function to obtain the weights on the values.

“Attention is all you need” paper [1]

Then we use the Q, K and V matrices to calculate the attention scores. The scores measure how much focus to place on other places or words of the input sequence w.r.t a word at a certain position. That is, the dot product of the query vector with the key vector of the respective word we’re scoring. So, for position 1 we calculate the dot product (.) of q1and k1, then q1. k2, q1. k3 and so on,…

Next we apply the “scaled” factor to have more stable gradients. The softmax function can not work properly with large values, resulting in vanishing the gradients and slowing down the learning, [2]. After “softmaxing” we multiply by the Value matrix to keep the values of the words we want to focus on and minimizing or removing the values for the irrelevant words (its value in V matrix should be very small).

The formula for these operations is:


From “Attention is all you need” paper by Vaswani, et al., 2017 [1]. Scaled dot-product attention formula.

Multi-head Attention
In the previous description the attention scores are focused on the whole sentence at a time, this would produce the same results even if two sentences contain the same words in a different order. Instead, we would like to attend to different segments of the words. “We can give the self attention greater power of discrimination, by combining several self attention heads, dividing the words vectors into a fixed number (h, number of heads) of chunks, and then self-attention is applied on the corresponding chunks, using Q, K and V sub-matrices.”, [2] Peter Bloem, “Transformers from scratch”. This produce h different output matrices of scores.


From “Attention is all you need” paper by Vaswani, et al., 2017 [1]
But the next layer (the Feed-Forward layer) is expecting just one matrix, a vector for each word, so “after calculating the dot product of every head, we concatenate the output matrices and multiply them by an additional weights matrix Wo,”[3]. This final matrix captures information from all the attention heads.


Positional Encoding
We mentioned briefly that the order of the words in the sentence is an issue to solve in this model, because the network and the self-attention mechanism is permutation invariant. If we shuffle up the words in the input sentence, we get the same solutions. We need to create a representation of the position of the word in the sentence and add it to the word embedding.

To this end, we add “positional encodings” to the input embeddings at the bottoms of the encoder and decoder stacks. The positional encodings have the same dimension as the embeddings, so that the two can be summed. There are many choices of positional encodings.

“Attention is all you need” paper

So, we apply a function to map the position in the sentence to a real valued vector. The network will learn how to use this information. Another approach would be to use a position embedding, similar to word embedding, coding every known position with a vector. “It would requiere sentences of all accepted positions during the training loop but positional encoding allow the model to extrapolate to sequence lengths longer than the ones encountered during training”, [2].

In the paper a sinusoidal function is applied:


From “Attention is all you need” paper by Vaswani, et al., 2017 [1]. Positional encoding

The encoder
Now that all the main pieces of the model have been described we can introduce the encoder components, [4]:

Positional encoding: Add the position encoding to the input embedding (our input words are transformed to embedding vectors). “The same weight matrix is shared between the two embedding layers (encoder and decoder) and the pre-softmax linear transformation. In the embedding layers, we multiply those weights by square root of the model dimension” [1].
N=6 identical layers, containing two sub-layers: a multi-head self-attention mechanism, and a fully connected feed-forward network (two linear transformations with a ReLU activation). But it is applied position-wise to the input, which means that the same neural network is applied to every single “token” vector belonging to the sentence sequence.

There is a residual connection around each sub-layer (attention and FC network), summing up the output of the layer with its input, followed by a layer normalization.
Before every residual connection, a regularization is applied: “We apply dropout to the output of each sub-layer, before it is added to the sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks” [1] with a dropout rate of 0.1.
Normalization and residual connections are standard tricks used to help deep neural networks train faster and more accurately. The layer normalization is applied over the embedding dimension only.

Peter Bloem, “Transformers from scratch” [2]

First we implement the encoder layer, each one of the six blocks, contained in an encoder:


The next figure will show the components detailed:


“The Ilustrated Transformer” by Jay Alammar [3]
And the encoder code:


Keep in mind that only the vector from the last layer (6-th) is sent to the decoder.

The Decoder
The decoder share some components with the encoder but they are used in a different way to take into account the encoder output, [4]:

Positional encoding: Similar that the one in the encoder
N=6 identical layers, containing 3 three sub-layers. First, the Masked Multi-head attention or masked causal attention to prevent positions from attending to subsequent positions. “This masking, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position i can depend only on the known outputs at positions less than i” [1]. It is implemented setting to −∞ the values corresponding to the forbidden states in the softmax layer of the dot-product attention modules. The second component or “encoder-decoder attention” performs multi-head attention over the output of the decoder, the Key and Value vectors come from the output of the encoder but the queries come from the previous decoder layer. “This allows every position in the decoder to attend over all positions in the input sequence” [1]. And finally the fully-connected network.
The residual connection and layer normalization around each sub-layer, similar to the encoder.
And repeat the same residual dropout that was executed in the encoder.
The decoder layer:



“The Ilustrated Transformer” by Jay Alammar [3]
At the end of the N stacked decoders, the linear layer, a fully-connected network, transforms the stacked outputs to a much larger vector, the logits. “The softmax layer then turns those scores (logits) into probabilities (all positive, all add up to 1.0). The cell with the highest probability is chosen, and the word associated with it is produced as the output for this time step”, [3] Jay Alammar, “The Ilustrated Transformer” .

The decoder component:


Joining all the pieces: the Transformer
Once we have defined our components and created the encoder, the decoder and the linear-softmax final layer, we join the pieces to form our model, the Transformer.

It is worth mentioning that we create 3 masks, each of which will allow us:

Encoder mask: It is a padding mask to discard the pad tokens from the attention calculation.
Decoder mask 1: this mask is a union of the padding mask and the look ahead mask which will help the causal attention to discard the tokens “in the future”. We take the maximum value between the padding mask and the look ahead one.
Decoder mask 2: it is the padding mask and is applied in the encoder-decoder attention layer.

As you can see, then we call the encoder, the decoder and the final linear-softmax layer to get the predicted output from our Transformer model.


Picture by Gerd Altmann From Pixabay
Training the Transformer model
Now that we have described in detail the components in the paper, we are ready to implement them and train a transformer model on a NMT problem. It is a toy problem for educational purposes.

We won’t deal with the data wrangling in this blog post. Follow the link I mentioned in the introduction for more information and the code provided to see how the data is loaded and prepared. In summary, create the vocabulary, tokenize (including a eos and sos token) and pad the sentences. Then we create a Dataset, a batch data generator, for training on batches.

We need to create a custom loss function to mask the padding tokens.


We use an Adam optimizer described in the paper, with beta1=0.9, beta2=0.98 and epsilon=10e-9 . And then we create a scheduler to vary the learning rate over the training process according to:


“Attention is all you need” paper. Learning rate decay.
The main train function
The train function is similar to many other Tensorflow trainings, an usual training loop for sequence-to-sequence tasks:
For every iteration on the batch generator that produce batch size inputs and outputs
Get the input sequence from 0 to length-1 and the actual outputs from 1 to length, the next word expected at every sequence step.
Call the transformer to get the predictions
Calculate the loss function between the real outputs and the predictions
Apply the gradients to update the weights in the model and update the optimizer too
Calculate the mean loss and the accuracy for the batch data
Show some results and save the model in every epoch
And that’s all, we have all the necessary elements to train our model, we just need to create them and call the train function:
Photo by Jr Korpa on Unsplash
Make prediction
When training a ML model we are not only interested in optimize losses or accuracies, we want our model to make good enough predictions and, in this case, see how the model works with new sentences. The predict function will input a tokenized sentence to the model and return the predicted new sentence, in our example, a translation from English to Spanish.

These are the steps in that process:
Tokenize the input sentence to a sequence of tokens
Set the initial output sequence to the sos token
Until we reach the max length or the eos token is returned by the model
Get the next word predicted. The model returns the logits, remember that the softmax function is applied in the loss calculation.
Get the index in the vocabulary of the word with the highest probability
Concatenate the next word predicted to the output sequence
And finally our last function receives a sentence in English, calls the transformer to translate it to Spanish and shows the result.
For this example, we just experiment with some values for the model dimension and the units of the feedforward network to train the model for an hour. If you want to optimize the model, you should probably train it for longer and with many different values for the hyperparameters.
The code is available in my github repository “Transformer-NMT”. The code is partially extracted from an excellent course by SuperDataScience Team called “Modern Natural Language Processing in Python” on Udemy. I highly recommend it.
Some examples of translations are:
#Show some translations
sentence = "you should pay for it."
print("Input sentence: {}".format(sentence))
predicted_sentence = translate(sentence)
print("Output sentence: {}".format(predicted_sentence))
Input sentence: you should pay for it. 
Output sentence: Deberías pagar por ello.
#Show some translations
sentence = "we have no extra money."
print("Input sentence: {}".format(sentence))
predicted_sentence = translate(sentence)
print("Output sentence: {}".format(predicted_sentence))
Input sentence: we have no extra money. 
Output sentence: No tenemos dinero extra.
#Show some translations
sentence = "This is a problem to deal with."
print("Input sentence: {}".format(sentence))
predicted_sentence = translate(sentence)
print("Output sentence: {}".format(predicted_sentence))
Input sentence: This is a problem to deal with. 
Output sentence: Este problema es un problema con eso.
I hope you enjoy experimenting with the Transformer model. In future post we will deal with another NLP tasks.
References
[1] Vaswani, Ashish & Shazeer, Noam & Parmar, Niki & Uszkoreit, Jakob & Jones, Llion & Gomez, Aidan & Kaiser, Lukasz & Polosukhin, Illia, “Attention is all you need” , 2017.
[2] Peter Bloem, “Transformers from scratch” blog post, 2019.
[3] Jay Alammar, “The Ilustrated Transformer” blog post, 2018.
[4] Lilian Weng, “Attention? Attention!!” blog post, 2018.
[5] Ricardo Faúndez-CArrasco, “Attention is all you need’s review” blog post, 2017
[6] Alexander Rush, “The Annotated Transformer”, 2018, Harvard NLP group.
NLP
TensorFlow
Machine Learning
Transformers
Getting Started
611
8
Eduardo Muñoz
Towards Data Science
Written by Eduardo Muñoz
498 Followers
·
Writer for 
Towards Data Science
A Data scientist and Machine Learning practitioner and involved in NLP tasks and advances. Experienced Project Management Lead. Learning every day.
Follow
More from Eduardo Muñoz and Towards Data Science
A Guide to the Encoder-Decoder Model and the Attention Mechanism
Eduardo Muñoz
Eduardo Muñoz
in
Better Programming
A Guide to the Encoder-Decoder Model and the Attention Mechanism
Create and train a neural machine translation model with attention in TF2
12 min read
Oct 11, 2020
248
The Math behind Adam Optimizer
Cristian Leo
Cristian Leo
in
Towards Data Science
The Math behind Adam Optimizer
Why is Adam the most popular optimizer in Deep Learning? Let’s understand it by diving into its math, and recreating the algorithm.
16 min read
·
Jan 30, 2024
1.93K
14
Python’s Most Powerful Decorator
Siavash Yasini
Siavash Yasini
in
Towards Data Science
Python’s Most Powerful Decorator
And 5 ways to use it in data science and machine learning
11 min read
·
Feb 2, 2024
2.2K
15
Create a Tokenizer and Train a Huggingface RoBERTa model from scratch
Eduardo Muñoz
Eduardo Muñoz
in
Analytics Vidhya
Create a Tokenizer and Train a Huggingface RoBERTa model from scratch
Part 1: A product names generator using an Encoder Decoder Transformer
7 min read
·
Aug 16, 2021
132
2
See all from Eduardo Muñoz
See all from Towards Data Science
Recommended from Medium
Illustration of the overall transformer architecture from “Attention is all you need”.
Stefan
Stefan
Understanding Attention and Transformers
My notes for understanding the attention mechanism and transformer architecture used by GPT-4 and other LLMs.
7 min read
·
Nov 28, 2023
52
Visualize attention scores of LLMs with BertViz
Gary Fan
Gary Fan
Visualize attention scores of LLMs with BertViz
BertViz is an interactive tool for visualizing attention in Transformer language models such as BERT, GPT2, or T5. It can be run inside a…
3 min read
·
Sep 7, 2023
6
2
Lists
Predictive Modeling w/ Python
20 stories
·
920 saves
Principal Component Analysis for ML
Time Series Analysis
deep learning cheatsheet for beginner
Practical Guides to Machine Learning
10 stories
·
1082 saves
Natural Language Processing
1215 stories
·
689 saves
Image by vectorjuice on FreePik
The New Chatbots: ChatGPT, Bard, and Beyond
12 stories
·
307 saves
Demystifying efficient self-attention
Thomas van Dongen
Thomas van Dongen
i
Towards Data Science

Demystifying efficient self-attention
A practical overview
20 min read
·
Nov 7, 2022
623
2
Intuition for Multi-headed Attention.
Ngieng Kianyew
Ngieng Kianyew

Intuition for Multi-headed Attention.
Follow up on the intuition on attention mechanism
7 min read
·
Sep 1, 2023
4
Transformer Architecture explained
Amanatullah
Amanatullah
Transformer Architecture explained
Transformers are a new development in machine learning that have been making a lot of noise lately. They are incredibly good at keeping…
10 min read
·
Sep 1, 2023
360
2
RNN vs. LSTM vs. Transformers: Unraveling the Secrets of Sequential Data Processing
Rokon
Rokon

RNN vs. LSTM vs. Transformers: Unraveling the Secrets of Sequential Data Processing
In the realm of deep learning, sequential data processing is at the heart of many applications, including natural language understanding…
3 min read
·
Sep 25, 2023
7
See more recommendations"""
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience

In [26]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 276 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (101, 32) into a new token 256
merging (32, 116) into a new token 257
merging (101, 110) into a new token 258
merging (104, 256) into a new token 259
merging (105, 110) into a new token 260
merging (32, 97) into a new token 261
merging (111, 110) into a new token 262
merging (101, 114) into a new token 263
merging (257, 259) into a new token 264
merging (111, 114) into a new token 265
merging (116, 105) into a new token 266
merging (115, 32) into a new token 267
merging (116, 32) into a new token 268
merging (266, 262) into a new token 269
merging (100, 32) into a new token 270
merging (97, 116) into a new token 271
merging (97, 110) into a new token 272
merging (111, 100) into a new token 273
merging (44, 32) into a new token 274
merging (114, 101) into a new token 275


In [27]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 21976
ids length: 16862
compression ratio: 1.30X


### Decoding
Given a sequence of integers in the range [0, vocab_size], what is the text?


In [28]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([128]))

�


### encoding

The other way around: Given a string, what are the tokens?


In [29]:
merges

{(101, 32): 256,
 (32, 116): 257,
 (101, 110): 258,
 (104, 256): 259,
 (105, 110): 260,
 (32, 97): 261,
 (111, 110): 262,
 (101, 114): 263,
 (257, 259): 264,
 (111, 114): 265,
 (116, 105): 266,
 (115, 32): 267,
 (116, 32): 268,
 (266, 262): 269,
 (100, 32): 270,
 (97, 116): 271,
 (97, 110): 272,
 (111, 100): 273,
 (44, 32): 274,
 (114, 101): 275}

In [30]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(encode(""))

[]


In [31]:
print(decode(encode("hello world")))

hello world


In [32]:
text2 = decode(encode(text))
print(text2 == text)

True


In [33]:
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes thousands of emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters."
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True


### Forced splits using regex patterns (GPT series)


In [49]:
import regex as re
gpt4pat = re.compile(r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "M      Hello've world123 how's are you!!!?"))

['M', '     ', ' Hello', "'ve", ' world', '123', ' how', "'s", ' are', ' you', '!!!?']


In [50]:
example = """
for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
"""
print(re.findall(gpt4pat, example))

['\n', 'for', ' i', ' in', ' range', '(', '1', ',', ' ', '101', '):\n', '   ', ' if', ' i', ' %', ' ', '3', ' ==', ' ', '0', ' and', ' i', ' %', ' ', '5', ' ==', ' ', '0', ':\n', '       ', ' print', '("', 'FizzBuzz', '")\n', '   ', ' elif', ' i', ' %', ' ', '3', ' ==', ' ', '0', ':\n', '       ', ' print', '("', 'Fizz', '")\n', '   ', ' elif', ' i', ' %', ' ', '5', ' ==', ' ', '0', ':\n', '       ', ' print', '("', 'Buzz', '")\n', '   ', ' else', ':\n', '       ', ' print', '(i', ')\n']


In [52]:
import tiktoken

# GPT-2 (does not merge spaces)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("    hello world!!!"))

# GPT-4 (merges spaces)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("    hello world!!!"))

[220, 220, 220, 23748, 995, 10185]
[262, 24748, 1917, 12340]


Reference the GPT-2 [encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
Download the vocab.bpe and encoder.json files.

In [53]:
#!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
#!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

In [54]:
import os, json

with open('encoder.json', 'r') as f:
    encoder = json.load(f) # <--- ~equivalent to our "vocab"

with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
# ^---- ~equivalent to our "merges"


### special tokens


In [55]:
len(encoder) # 256 raw byte tokens. 50,000 merges. +1 special token

50257

In [56]:
encoder['<|endoftext|>'] # the only special token in use for the GPT-2 base model

50256

In [57]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # GPT-4 tokenizer
print(enc.encode("안녕하세요 👋 (hello in Korean!)"))
print(enc.decode(enc.encode("안녕하세요 👋 (hello in Korean!)")) == "안녕하세요 👋 (hello in Korean!)")
# match the above for your own tokenizer, and also implement a train() function

[31495, 230, 75265, 243, 92245, 62904, 233, 320, 15339, 304, 16526, 16715]
True


### sentencepiece

Commonly used because (unlike tiktoken) it can efficiently both train and inference BPE tokenizers. It is used in both Llama and Mistral series.

[sentencepiece on Github link](https://github.com/google/sentencepiece).

**The big difference**: sentencepiece runs BPE on the Unicode code points directly! It then has an option `character_coverage` for what to do with very very rare codepoints that appear very few times, and it either maps them onto an UNK token, or if `byte_fallback` is turned on, it encodes them with utf-8 and then encodes the raw bytes instead.

TLDR:

- tiktoken encodes to utf-8 and then BPEs bytes
- sentencepiece BPEs the code points and optionally falls back to utf-8 bytes for rare code points (rarity is determined by character_coverage hyperparameter), which then get translated to byte tokens.

(Personally I think the tiktoken way is a lot cleaner...)

In [59]:
import sentencepiece as spm

In [60]:
# write a toy.txt file with some random text
with open("toy.txt", "w", encoding="utf-8") as f:
  f.write("SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.")

Docs for sentencepiece options:

- [markdown](https://github.com/google/sentencepiece/blob/master/doc/options.md)
- [protobuf](https://github.com/google/sentencepiece/blob/master/src/sentencepiece_model.proto#L193)

In [ ]:
# train a sentencepiece model on it
# the settings here are (best effort) those used for training Llama 2
import os

options = dict(
  # input spec
  input="toy.txt",
  input_format="text",
  # output spec
  model_prefix="tok400", # output filename prefix
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=400,
  # normalization
  normalization_rule_name="identity", # ew, turn off normalization
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=True,
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)

spm.SentencePieceTrainer.train(**options)


In [63]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [64]:
ids = sp.encode("hello 안녕하세요")
print(ids)

[362, 378, 361, 372, 358, 362, 239, 152, 139, 238, 136, 152, 240, 152, 155, 239, 135, 187, 239, 157, 151]


In [65]:
print([sp.id_to_piece(idx) for idx in ids])

['▁', 'h', 'e', 'l', 'lo', '▁', '<0xEC>', '<0x95>', '<0x88>', '<0xEB>', '<0x85>', '<0x95>', '<0xED>', '<0x95>', '<0x98>', '<0xEC>', '<0x84>', '<0xB8>', '<0xEC>', '<0x9A>', '<0x94>']


**Llama 2 tokenizer proto**
If you'd like to export the raw protocol buffer for the `tokenizer.model` released by meta, this is a [helpful issue](https://github.com/google/sentencepiece/issues/121). And this is the result:

```
normalizer_spec {
  name: "identity"
  precompiled_charsmap: ""
  add_dummy_prefix: true
  remove_extra_whitespaces: false
  normalization_rule_tsv: ""
}

trainer_spec {
  input: "/large_experiments/theorem/datasets/MERGED/all.test1.merged"
  model_prefix: "spm_model_32k_200M_charcov099995_allowWSO__v2"
  model_type: BPE
  vocab_size: 32000
  self_test_sample_size: 0
  input_format: "text"
  character_coverage: 0.99995
  input_sentence_size: 200000000
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  num_threads: 80
  num_sub_iterations: 2
  max_sentence_length: 4192
  shuffle_input_sentence: true
  max_sentencepiece_length: 16
  split_by_unicode_script: true
  split_by_whitespace: true
  split_by_number: true
  treat_whitespace_as_suffix: false
  split_digits: true
  allow_whitespace_only_pieces: true
  vocabulary_output_piece_score: true
  hard_vocab_limit: true
  use_all_vocab: false
  byte_fallback: true
  required_chars: ""
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_surface: " \342\201\207 "
  unk_piece: "<unk>"
  bos_piece: "<s>"
  eos_piece: "</s>"
  pad_piece: "<pad>"
  train_extremely_large_corpus: false
  enable_differential_privacy: false
  differential_privacy_noise_level: 0.0
  differential_privacy_clipping_threshold: 0
}
```